In [ ]:
import os
import cv2
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt
import random
from keras.utils import normalize
from focal_loss import BinaryFocalLoss
from unetmodel import build_unet
from keras.optimizers import Adam
from keras import backend as K
import tensorflow as tf
from skimage.transform import resize


IMG_HEIGHT = 256
IMG_WIDTH  = 256
IMG_CHANNELS = 3

#def load_data():
image_directory = 'data/train/images/'
mask_directory = 'data/train/masks/'

Test_imageDir = 'data/test/images/'
mask_directory = 'data/train/masks/'

X_train = np.zeros((len(os.listdir(image_directory)),IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS  ), dtype=np.uint8)
Y_train = np.zeros((len(os.listdir(image_directory)),IMG_HEIGHT, IMG_WIDTH, 1  ), dtype=bool)

print(X_train.shape)
print(Y_train.shape)

Train_images = os.listdir(image_directory)
for i, image_name in enumerate(Train_images):
    image = cv2.imread(image_directory+image_name, 1)
    X_train[i] = image

mask_files = os.listdir(mask_directory)
Y_train = []  # Initialize Y_train as an empty list
mask = np.zeros((IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.uint8)
for i, mask_file in enumerate(mask_files):
    mask_path = os.path.join(mask_directory, mask_file)
    mask_ = cv2.imread(mask_path, 0)
    mask_ = np.expand_dims(resize(mask_, (IMG_HEIGHT, IMG_WIDTH), mode='constant',
                                      preserve_range=True), axis=-1)
    mask = np.maximum(mask, mask_)
    if (i + 1) % 6 == 0:
        y = int((i + 1) / 6)  # Convert y to an integer
        Y_train.append(mask.copy())  # Append a copy of the accumulated mask to Y_train
        mask = np.zeros((IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.uint8)

Y_train = np.array(Y_train)
print("X Train")
print(len(X_train))
print(X_train.shape)
print(X_train[1].shape)
print(X_train[1].astype)

print("Y Train")
print(len(Y_train))
print(Y_train.shape)
print(Y_train[1].shape)
print(Y_train[1].astype)


#X_test
X_Test = np.zeros((len(os.listdir(Test_imageDir)),IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS  ), dtype=np.uint8)
Test_images = os.listdir(Test_imageDir)
for i, image_name in enumerate(Test_images):
    image = cv2.imread(Test_imageDir+image_name, 1)
    X_Test[i] = image
print(X_Test[1].shape)

# Function to plot random images and their masks
def plot_random_images_masks(X_train, Y_train, num_images=4):
    indices = np.random.choice(len(X_train), num_images, replace=False)
    plt.figure(figsize=(15, 5*num_images))

    for i, idx in enumerate(indices):
        image = X_train[idx]
        mask = Y_train[idx]

        plt.subplot(num_images, 2, 2*i + 1)
        plt.imshow(image)
        plt.title('Image')
        plt.axis('off')

        plt.subplot(num_images, 2, 2*i + 2)
        plt.imshow(mask.squeeze(), cmap='gray')
        plt.title('Mask')
        plt.axis('off')

    plt.tight_layout()
    plt.show()

# Plot random images and their masks
#plot_random_images_masks(X_train, Y_train)


# Build the model
inputs = tf.keras.layers.Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
s = tf.keras.layers.Lambda(lambda x: x / 255)(inputs)

# Contraction path
c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(s)
c1 = tf.keras.layers.Dropout(0.1)(c1)
c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
c2 = tf.keras.layers.Dropout(0.1)(c2)
c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)

c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
c3 = tf.keras.layers.Dropout(0.2)(c3)
c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)

c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
c4 = tf.keras.layers.Dropout(0.2)(c4)
c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c4)

c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
c5 = tf.keras.layers.Dropout(0.3)(c5)
c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)

# Expansive path
u6 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
u6 = tf.keras.layers.concatenate([u6, c4])
c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
c6 = tf.keras.layers.Dropout(0.2)(c6)
c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)

u7 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
u7 = tf.keras.layers.concatenate([u7, c3])
c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
c7 = tf.keras.layers.Dropout(0.2)(c7)
c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)

u8 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
u8 = tf.keras.layers.concatenate([u8, c2])
c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
c8 = tf.keras.layers.Dropout(0.1)(c8)
c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)

u9 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
u9 = tf.keras.layers.concatenate([u9, c1], axis=3)
c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
c9 = tf.keras.layers.Dropout(0.1)(c9)
c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)

outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(c9)

model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


################################
#Modelcheckpoint
checkpointer = tf.keras.callbacks.ModelCheckpoint('model_for_nuclei.keras', verbose=1, save_best_only=True)

callbacks = [
        tf.keras.callbacks.EarlyStopping(patience=3, monitor='val_loss'),
        tf.keras.callbacks.TensorBoard(log_dir='logs')]

results = model.fit(X_train, Y_train, validation_split=0.1, batch_size=16, epochs=100, callbacks=callbacks)

####################################

